# 0005.0007 Modeling Strategy - Aggregate and Cut Transformed Features

In [1]:
import pathlib
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.mlab as mlab

%matplotlib inline

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
#add the 'src' directory to path to import modules
PROJECT_DIR = pathlib.Path.cwd().resolve().parent
sys.path.append(str(PROJECT_DIR))


pd.set_option('display.max_columns', 1500)
pd.set_option('display.width', 1000)

DATA_DIR = PROJECT_DIR / 'data'
# SCOPED_DATA_DIR = DATA_DIR / '03-scoped'
# PROCESSED_DATA_DIR = DATA_DIR / '04-processed'
SEMI_MERGED_DIR = DATA_DIR / '05-processed-semi-merged'

### Forming Historical Data based on Game Features

In [3]:
load_fp = SEMI_MERGED_DIR / 'english-premier-league' / str('english-premier-league' + '.csv')
df_load = pd.read_csv(load_fp, parse_dates=['date'], index_col=None)
df_load.head()

,season,date,h,a,h_corners,a_corners,h_h_corners_-1,h_h_corners_-10,h_h_corners_-11,h_h_corners_-12,h_h_corners_-13,h_h_corners_-14,h_h_corners_-15,h_h_corners_-16,h_h_corners_-17,h_h_corners_-18,h_h_corners_-2,h_h_corners_-3,h_h_corners_-4,h_h_corners_-5,h_h_corners_-6,h_h_corners_-7,h_h_corners_-8,h_h_corners_-9,a_a_corners_-1,a_a_corners_-10,a_a_corners_-11,a_a_corners_-12,a_a_corners_-13,a_a_corners_-14,a_a_corners_-15,a_a_corners_-16,a_a_corners_-17,a_a_corners_-18,a_a_corners_-2,a_a_corners_-3,a_a_corners_-4,a_a_corners_-5,a_a_corners_-6,a_a_corners_-7,a_a_corners_-8,a_a_corners_-9,a_h_corners_-1,a_h_corners_-2,a_h_corners_-11,a_h_corners_-12,a_h_corners_-13,a_h_corners_-14,a_h_corners_-15,a_h_corners_-16,a_h_corners_-17,a_h_corners_-18,a_h_corners_-19,a_h_corners_-3,a_h_corners_-4,a_h_corners_-5,a_h_corners_-6,a_h_corners_-7,a_h_corners_-8,a_h_corners_-9,a_h_corners_-10,h_a_corners_-1,h_a_corners_-2,h_a_corners_-11,h_a_corners_-12,h_a_corners_-13,h_a_corners_-14,h_a_corners_-15,h_a_corners_-16,h_a_corners_-17,h_a_corners_-18,h_a_corners_-19,h_a_corners_-3,h_a_corners_-4,h_a_corners_-5,h_a_corners_-6,h_a_corners_-7,h_a_corners_-8,h_a_corners_-9,h_a_corners_-10,h_drawoddsbbmean,a_drawoddsbbmean,h_h_drawoddsbbmean_-1,h_h_drawoddsbbmean_-10,h_h_drawoddsbbmean_-11,h_h_drawoddsbbmean_-12,h_h_drawoddsbbmean_-13,h_h_drawoddsbbmean_-14,h_h_drawoddsbbmean_-15,h_h_drawoddsbbmean_-16,h_h_drawoddsbbmean_-17,h_h_drawoddsbbmean_-18,h_h_drawoddsbbmean_-2,h_h_drawoddsbbmean_-3,h_h_drawoddsbbmean_-4,h_h_drawoddsbbmean_-5,h_h_drawoddsbbmean_-6,h_h_drawoddsbbmean_-7,h_h_drawoddsbbmean_-8,h_h_drawoddsbbmean_-9,a_a_drawoddsbbmean_-1,a_a_drawoddsbbmean_-10,a_a_drawoddsbbmean_-11,a_a_drawoddsbbmean_-12,a_a_drawoddsbbmean_-13,a_a_drawoddsbbmean_-14,a_a_drawoddsbbmean_-15,a_a_drawoddsbbmean_-16,a_a_drawoddsbbmean_-17,a_a_drawoddsbbmean_-18,a_a_drawoddsbbmean_-2,a_a_drawoddsbbmean_-3,a_a_drawoddsbbmean_-4,a_a_drawoddsbbmean_-5,a_a_drawoddsbbmean_-6,a_a_drawoddsbbmean_-7,a_a_drawoddsbbmean_-8,a_a_drawoddsbbmean_-9,a_h_drawoddsbbmean_-1,a_h_drawoddsbbmean_-2,a_h_drawoddsbbmean_-11,a_h_drawoddsbbmean_-12,a_h_drawoddsbbmean_-13,a_h_drawoddsbbmean_-14,a_h_drawoddsbbmean_-15,a_h_drawoddsbbmean_-16,a_h_drawoddsbbmean_-17,a_h_drawoddsbbmean_-18,a_h_drawoddsbbmean_-19,a_h_drawoddsbbmean_-3,a_h_drawoddsbbmean_-4,a_h_drawoddsbbmean_-5,a_h_drawoddsbbmean_-6,a_h_drawoddsbbmean_-7,a_h_drawoddsbbmean_-8,a_h_drawoddsbbmean_-9,a_h_drawoddsbbmean_-10,h_a_drawoddsbbmean_-1,h_a_drawoddsbbmean_-2,h_a_drawoddsbbmean_-11,h_a_drawoddsbbmean_-12,h_a_drawoddsbbmean_-13,h_a_drawoddsbbmean_-14,h_a_drawoddsbbmean_-15,h_a_drawoddsbbmean_-16,h_a_drawoddsbbmean_-17,h_a_drawoddsbbmean_-18,h_a_drawoddsbbmean_-19,h_a_drawoddsbbmean_-3,h_a_drawoddsbbmean_-4,h_a_drawoddsbbmean_-5,h_a_drawoddsbbmean_-6,h_a_drawoddsbbmean_-7,h_a_drawoddsbbmean_-8,h_a_drawoddsbbmean_-9,h_a_drawoddsbbmean_-10,h_fouls,a_fouls,h_h_fouls_-1,h_h_fouls_-10,h_h_fouls_-11,h_h_fouls_-12,h_h_fouls_-13,h_h_fouls_-14,h_h_fouls_-15,h_h_fouls_-16,h_h_fouls_-17,h_h_fouls_-18,h_h_fouls_-2,h_h_fouls_-3,h_h_fouls_-4,h_h_fouls_-5,h_h_fouls_-6,h_h_fouls_-7,h_h_fouls_-8,h_h_fouls_-9,a_a_fouls_-1,a_a_fouls_-10,a_a_fouls_-11,a_a_fouls_-12,a_a_fouls_-13,a_a_fouls_-14,a_a_fouls_-15,a_a_fouls_-16,a_a_fouls_-17,a_a_fouls_-18,a_a_fouls_-2,a_a_fouls_-3,a_a_fouls_-4,a_a_fouls_-5,a_a_fouls_-6,a_a_fouls_-7,a_a_fouls_-8,a_a_fouls_-9,a_h_fouls_-1,a_h_fouls_-2,a_h_fouls_-11,a_h_fouls_-12,a_h_fouls_-13,a_h_fouls_-14,a_h_fouls_-15,a_h_fouls_-16,a_h_fouls_-17,a_h_fouls_-18,a_h_fouls_-19,a_h_fouls_-3,a_h_fouls_-4,a_h_fouls_-5,a_h_fouls_-6,a_h_fouls_-7,a_h_fouls_-8,a_h_fouls_-9,a_h_fouls_-10,h_a_fouls_-1,h_a_fouls_-2,h_a_fouls_-11,h_a_fouls_-12,h_a_fouls_-13,h_a_fouls_-14,h_a_fouls_-15,h_a_fouls_-16,h_a_fouls_-17,h_a_fouls_-18,h_a_fouls_-19,h_a_fouls_-3,h_a_fouls_-4,h_a_fouls_-5,h_a_fouls_-6,h_a_fouls_-7,h_a_fouls_-8,h_a_fouls_-9,h_a_fouls_-10,h_ftgoals,a_ftgoals,h_h_ftgoals_-1,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,

In [4]:

df_load.tail()

,season,date,h,a,h_corners,a_corners,h_h_corners_-1,h_h_corners_-10,h_h_corners_-11,h_h_corners_-12,h_h_corners_-13,h_h_corners_-14,h_h_corners_-15,h_h_corners_-16,h_h_corners_-17,h_h_corners_-18,h_h_corners_-2,h_h_corners_-3,h_h_corners_-4,h_h_corners_-5,h_h_corners_-6,h_h_corners_-7,h_h_corners_-8,h_h_corners_-9,a_a_corners_-1,a_a_corners_-10,a_a_corners_-11,a_a_corners_-12,a_a_corners_-13,a_a_corners_-14,a_a_corners_-15,a_a_corners_-16,a_a_corners_-17,a_a_corners_-18,a_a_corners_-2,a_a_corners_-3,a_a_corners_-4,a_a_corners_-5,a_a_corners_-6,a_a_corners_-7,a_a_corners_-8,a_a_corners_-9,a_h_corners_-1,a_h_corners_-2,a_h_corners_-11,a_h_corners_-12,a_h_corners_-13,a_h_corners_-14,a_h_corners_-15,a_h_corners_-16,a_h_corners_-17,a_h_corners_-18,a_h_corners_-19,a_h_corners_-3,a_h_corners_-4,a_h_corners_-5,a_h_corners_-6,a_h_corners_-7,a_h_corners_-8,a_h_corners_-9,a_h_corners_-10,h_a_corners_-1,h_a_corners_-2,h_a_corners_-11,h_a_corners_-12,h_a_corners_-13,h_a_corners_-14,h_a_corners_-15,h_a_corners_-16,h_a_corners_-17,h_a_corners_-18,h_a_corners_-19,h_a_corners_-3,h_a_corners_-4,h_a_corners_-5,h_a_corners_-6,h_a_corners_-7,h_a_corners_-8,h_a_corners_-9,h_a_corners_-10,h_drawoddsbbmean,a_drawoddsbbmean,h_h_drawoddsbbmean_-1,h_h_drawoddsbbmean_-10,h_h_drawoddsbbmean_-11,h_h_drawoddsbbmean_-12,h_h_drawoddsbbmean_-13,h_h_drawoddsbbmean_-14,h_h_drawoddsbbmean_-15,h_h_drawoddsbbmean_-16,h_h_drawoddsbbmean_-17,h_h_drawoddsbbmean_-18,h_h_drawoddsbbmean_-2,h_h_drawoddsbbmean_-3,h_h_drawoddsbbmean_-4,h_h_drawoddsbbmean_-5,h_h_drawoddsbbmean_-6,h_h_drawoddsbbmean_-7,h_h_drawoddsbbmean_-8,h_h_drawoddsbbmean_-9,a_a_drawoddsbbmean_-1,a_a_drawoddsbbmean_-10,a_a_drawoddsbbmean_-11,a_a_drawoddsbbmean_-12,a_a_drawoddsbbmean_-13,a_a_drawoddsbbmean_-14,a_a_drawoddsbbmean_-15,a_a_drawoddsbbmean_-16,a_a_drawoddsbbmean_-17,a_a_drawoddsbbmean_-18,a_a_drawoddsbbmean_-2,a_a_drawoddsbbmean_-3,a_a_drawoddsbbmean_-4,a_a_drawoddsbbmean_-5,a_a_drawoddsbbmean_-6,a_a_drawoddsbbmean_-7,a_a_drawoddsbbmean_-8,a_a_drawoddsbbmean_-9,a_h_drawoddsbbmean_-1,a_h_drawoddsbbmean_-2,a_h_drawoddsbbmean_-11,a_h_drawoddsbbmean_-12,a_h_drawoddsbbmean_-13,a_h_drawoddsbbmean_-14,a_h_drawoddsbbmean_-15,a_h_drawoddsbbmean_-16,a_h_drawoddsbbmean_-17,a_h_drawoddsbbmean_-18,a_h_drawoddsbbmean_-19,a_h_drawoddsbbmean_-3,a_h_drawoddsbbmean_-4,a_h_drawoddsbbmean_-5,a_h_drawoddsbbmean_-6,a_h_drawoddsbbmean_-7,a_h_drawoddsbbmean_-8,a_h_drawoddsbbmean_-9,a_h_drawoddsbbmean_-10,h_a_drawoddsbbmean_-1,h_a_drawoddsbbmean_-2,h_a_drawoddsbbmean_-11,h_a_drawoddsbbmean_-12,h_a_drawoddsbbmean_-13,h_a_drawoddsbbmean_-14,h_a_drawoddsbbmean_-15,h_a_drawoddsbbmean_-16,h_a_drawoddsbbmean_-17,h_a_drawoddsbbmean_-18,h_a_drawoddsbbmean_-19,h_a_drawoddsbbmean_-3,h_a_drawoddsbbmean_-4,h_a_drawoddsbbmean_-5,h_a_drawoddsbbmean_-6,h_a_drawoddsbbmean_-7,h_a_drawoddsbbmean_-8,h_a_drawoddsbbmean_-9,h_a_drawoddsbbmean_-10,h_fouls,a_fouls,h_h_fouls_-1,h_h_fouls_-10,h_h_fouls_-11,h_h_fouls_-12,h_h_fouls_-13,h_h_fouls_-14,h_h_fouls_-15,h_h_fouls_-16,h_h_fouls_-17,h_h_fouls_-18,h_h_fouls_-2,h_h_fouls_-3,h_h_fouls_-4,h_h_fouls_-5,h_h_fouls_-6,h_h_fouls_-7,h_h_fouls_-8,h_h_fouls_-9,a_a_fouls_-1,a_a_fouls_-10,a_a_fouls_-11,a_a_fouls_-12,a_a_fouls_-13,a_a_fouls_-14,a_a_fouls_-15,a_a_fouls_-16,a_a_fouls_-17,a_a_fouls_-18,a_a_fouls_-2,a_a_fouls_-3,a_a_fouls_-4,a_a_fouls_-5,a_a_fouls_-6,a_a_fouls_-7,a_a_fouls_-8,a_a_fouls_-9,a_h_fouls_-1,a_h_fouls_-2,a_h_fouls_-11,a_h_fouls_-12,a_h_fouls_-13,a_h_fouls_-14,a_h_fouls_-15,a_h_fouls_-16,a_h_fouls_-17,a_h_fouls_-18,a_h_fouls_-19,a_h_fouls_-3,a_h_fouls_-4,a_h_fouls_-5,a_h_fouls_-6,a_h_fouls_-7,a_h_fouls_-8,a_h_fouls_-9,a_h_fouls_-10,h_a_fouls_-1,h_a_fouls_-2,h_a_fouls_-11,h_a_fouls_-12,h_a_fouls_-13,h_a_fouls_-14,h_a_fouls_-15,h_a_fouls_-16,h_a_fouls_-17,h_a_fouls_-18,h_a_fouls_-19,h_a_fouls_-3,h_a_fouls_-4,h_a_fouls_-5,h_a_fouls_-6,h_a_fouls_-7,h_a_fouls_-8,h_a_fouls_-9,h_a_fouls_-10,h_ftgoals,a_ftgoals,h_h_ftgoals_-1,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,

In [ ]:
def get_feature_list()

In [ ]:
stop

## Functions to Cut Data to Defined Shape prior to PreProcessing

In [ ]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(past_games+1) \
               or col.isalpha() \
               or 'odds' in col]
    keeper_cols = keeper_cols + ['result']#, 'h_ftgoals', 'a_ftgoals']
    
#     # Rename the target columns to start with prefix 'res_'
#     res_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
#                 and int(col.split('_')[-1]) == 0]
#     res_cols.append('result')
#     res_cols = [col.replace('0', 'res') for col in res_cols]
#     print(res_cols)
#     print(res_cols)
    df = df[keeper_cols]
    return df

df_load = pd.read_csv(save_fp, parse_dates = ['date'], index_col=None) 
df_cut = get_n_past_games(df_load)
# # Restrict Historical records to last 4 games per team
# df_cut = df_load.copy(deep=True)
# hist_games = 2
# keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
#                and int(col.split('_')[-1]) > -(hist_games+1) \
#                and int(col.split('_')[-1]) < 0 \
#                or col.isalpha() \
#                or 'odds' in col]
# df_cut = df_cut[keeper_cols]
df_cut.head()

In [ ]:
df_cut.tail()

In [ ]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
    

# Restrict Historical records to last 4 games per team
df_cut = df_load.copy(deep=True)
hist_games = 2
keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
df_cut.head(10)